# Chapter 7. Running on a Cluster

### Introduction

> 앞서 spark-shell과 Spark local mode를 이용해 spark의 여러 동작을 배워봤다.  
사실 Spark를 사용하는 이유는 연산의 규모에 따라서 클러스터의 규모를 유동적으로 늘리거나 줄여서 효과적인 작업을 수행하기 위함이다.   
Spark의 장점은 사용자가 이러한 scaling에 대해서 크게 신경쓰지 않아도 된다는 점 이다. 

> 이 장에서는 아래 내용을 배우게 될 것이다.
> - runtime **architecture** of a distributed Spark application
> - **options** for running Spark in distributed clusters.
> - variety of **cluster managers** (Hadoop YARN, Apache Mesos and built-in Standalone cluster manager)
> - scheduling, deploying, and configuration a Spark application.

### Spark Runtime Architecture

> 분산환경에서 Spark는 master / slave 구조를 동작함  

> master에 해당하는 것이 **driver**이고, slave혹은 worker에 해당하는 것이 **executor**이다.  
driver과 각 executor는 각각의 Java process로 실행된다.   
driver와 이 driver과 동작하는 executor들을 **Spark application**이라 한다.

> <u>cluster manager</u>는 Spark application이 여러 머신에서 동작할 수 있도록 해 준다.   
Spark에는 기본적으로 **Standalone cluster manager**가 내장되어 있고, Hadoop YARN이나 Apache Mesos를 사용할수도 있다.

 ![Local image](./_images/figure_7_1.PNG "Tooltip for local image")

###### The Driver

> **driver**는 사용자 프로그램의 main() 메소드가 실행되는 프로세스이다.  
이 프로세스에서 SparkContext를 만들고, RDD를 만들고 transformation과 action을 수행하는 등의 코드가 실행된다.  
driver가 종료되면 해당 어플리케이션이 종료된다.   


> driver는 두가지 일을 한다.  
> - <i>사용자 프로그램을 task들로 변환</i>: Spark program은 논리적인 실행 계획인 DAG(directed acylic graph)를 생성한다. 그리고 driver가 실행되면서 이 논리적 실행계획을 물리적인 실행계획으로 변환한다. 그 과정에서 여러가지 최적화가 진행되고 각 실행단계를 여러개의 stage들로 변환한다. 또한 각 stage는 여러개의 task들로 나뉘게 된다.

> - <i>task들을 각 executor들에 스케쥴링</i>: 각 executor들은 task를 수행하고 RDD를 저장하는 프로세스인데, 이들은 driver에 등록되어 있고, driver는 각 task들을 적절한 executor들에 배분하게 된다. 

###### Executors

> **executor**는 task들을 실행하고 그 결과를 driver에 반환하고, RDD를 메모리에 저장하는 역할을 한다.

> (참고) 책에서의 예제는 모두 Spark local mode에서 동작하고 있다. 이 모드에서는 driver와 executor가 하나의 Java process에서 동작하게 된다. 그러나 일반적인 경우 driver와 executor는 독립적인 process에서 실행된다.

###### Cluster Manager

> 그렇다면 최초에 executor들은 어떻게 생성될까? Spark는 이러한 일들을 cluster manager에 맡기고 있다. Spark에서 사용 가능한 cluster manager에는 (내장) Standalone cluster manager, YARN, Mesos가 있다.

> (참고) driver, executor라는 용어와 더불어 master, worker라는 용어가 있는데 이들을 혼동하기 쉽다. master는 Hadoop YARN의 Resource Manager처럼 cluster manager의 중심부를 의미하며, worker는 Node Manager와 같이 cluster manager의 분산된 부분을 의미한다. 사실 Spark의 driver와 executor 모두 YARN의 worker node에서 실행된다.

###### Launching a Program

> 어떤 cluster manager를 사용하건 간에 Spark에 프로그램을 전달하기 위해 **spark-submit**이라는 script를 사용할 수 있다.
  
> 사실 어떤 cluster manager(YARN 같은)에서는 driver를 cluster내에 실행할 수도 있다. 물론 다른 cluster manager들은 local machine에 driver를 실행한다.

###### Summary

> Spark application이 cluster에서 실행되는 과정은 아래와 같다.

> 1. spark-submit을 이용해 사용자 프로그램을 전달한다.
> 2. spark-submit은 driver program을 띄우고, 제출한 프로그램의 main() 메소드를 실행한다.
> 3. driver 프로그램은 cluster manager에 가용한 자원이 있는지 질의한다.
> 4. cluster manager는 driver대신  executor들을 띄운다.
> 5. driver는 사용자 프로그램을 실행하면서 RDD transformation과 action에 따라 task들은 executor들에 보낸다.
> 6. executor들은 task들을 수행하고 결과들을 저장한다.
> 7. driver의 main() 메소드가 다 실행되어 종료되거나 **SparkContext.stop()**이 실행되면, executor들 또한 종료되고 자원을 cluster manager에 반환한다.

### Deploying Applications with spark-submit

### Packaging Your Code and Dependencies

### Scheduling Your Code and Dependencies

### Cluster Managers

### With Cluster Manager to Use?

### Conclusion